# Census Data

This notebook imports 4 csv files that contain census data for the 207 tracts in Suffolk County, MA. The data has been normalized (credit to Brown University) to reflect the tracts from the 2010 census. After a number of pre-processing steps, I calculate the percentage change between the two censuses with an end goal of using k-means clustering on the tracts.

In [1]:
import pandas as pd

In [203]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

## Data pre-processing (2000 census, full count)

In [408]:
df_2000 = pd.read_csv('LTDB_Std_2000_fullcount.csv',sep=',', engine='python')

In [409]:
df_MA_full_2000 = df_2000[(df_2000['state']=='MA') & (df_2000['county']=='Suffolk County')]

In [410]:
df_MA_full_2000.drop(columns=['placefp10','cbsa10','metdiv10','ccflag10'],axis=1,inplace=True)

In [411]:
df_MA_full_2000.reset_index(inplace=True, drop=True)

In [412]:
# rename tract id column to match 2000
df_MA_full_2000.rename(columns={'TRTID10':'tractid'},inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [413]:
# make columns in same order as 2010 so we can iterate over the df
df_MA_full_2000 = df_MA_full_2000[['tractid', 'state', 'county', 'tract', 'POP00', 'NHWHT00', 'NHBLK00',
       'NTV00', 'ASIAN00', 'HISP00', 'HAW00', 'INDIA00', 'CHINA00', 'FILIP00',
       'JAPAN00', 'KOREA00', 'VIET00', 'MEX00', 'PR00', 'CUBAN00', 'FAMILY00','FHH00','HU00',
       'VAC00', 'OHU00','OWN00', 'RENT00', 'A18UND00', 'A60UP00', 'A75UP00', 'AGEWHT00',
       'A15WHT00', 'A60WHT00', 'AGEBLK00', 'A15BLK00', 'A60BLK00', 'AGEHSP00',
       'A15HSP00', 'A60HSP00', 'AGEASN00','A15ASN00','A60ASN00','AGENTV00','A15NTV00', 'A60NTV00',
       'GlobD00', 'GlobG00']]

In [414]:
df_MA_full_2000[df_MA_full_2000['tractid']==25025981800]

,tractid,state,county,tract,POP00,NHWHT00,NHBLK00,NTV00,ASIAN00,HISP00,HAW00,INDIA00,CHINA00,FILIP00,JAPAN00,KOREA00,VIET00,MEX00,PR00,CUBAN00,FAMILY00,FHH00,HU00,VAC00,OHU00,OWN00,RENT00,A18UND00,A60UP00,A75UP00,AGEWHT00,A15WHT00,A60WHT00,AGEBLK00,A15BLK00,A60BLK00,AGEHSP00,A15HSP00,A60HSP00,AGEASN00,A15ASN00,A60ASN00,AGENTV00,A15NTV00,A60NTV00,GlobD00,GlobG00
202,25025981800,MA,Suffolk County,Census Tract 9818,5.011266,4.136201,0.239674,0.013373,0.308941,0.276137,0.000366,0.059378,0.119262,0.016541,0.049075,0.034982,0.009474,0.041963,0.071512,0.029322,1.046647,0.041287,2.364758,0.062389,2.302369,1.378303,0.924067,0.65182,0.987342,0.423244,4.136201,0.452929,0.916435,0.213941,0.038331,0.023326,0.276137,0.050268,0.021383,0.285644,0.028801,0.016513,0.004265,0.001126,0.0,wba,Semi global


## Data pre-processing (2010 census, full count)

In [415]:
df_2010 = pd.read_csv('LTDB_Std_2010_fullcount.csv',sep=',', engine='python')

In [416]:
df_MA_full_2010 = df_2010[(df_2010['state']=='MA') & (df_2010['county']=='Suffolk County')]

In [417]:
df_MA_full_2010.reset_index(inplace=True,drop=True)

## Calculate change b/w censuses (full count)

In [166]:
## New df for change in values between 2000 and 2010

In [418]:
df_joined = df_MA_full_2000.merge(df_MA_full_2010, on='tractid')

In [419]:
df_joined.drop(columns=['state_y','county_y','tract_y'],inplace=True)

In [420]:
df_joined.drop(columns=['GlobD00','GlobG00','GlobD10','GlobG10'],axis=1,inplace=True)

In [429]:
# for 
i = 4
j = 45
# stop once you get to final column
while j < 86:
    df_joined[f'{df_joined.columns[i]}_POC'] = df_joined.iloc[:,4:].apply(lambda x: 100*
                                    ((x[df_joined.columns[j]]-x[df_joined.columns[i]])
                                     /x[df_joined.columns[i]]),axis=1)
    i +=1
    j +=1

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [227]:
df_joined.to_csv('Full_count_census_changes.csv')

## Data pre-processing (2000 Census, Sample count)

In [484]:
df_2000_2 = pd.read_csv('LTDB_Std_2000_Sample.csv',sep=',', engine='python')

In [485]:
df_MA_sample_2000 = df_2000_2[(df_2000_2['state']=='MA') & (df_2000_2['county']=='Suffolk County')]

In [486]:
df_MA_sample_2000.drop(columns=['placefp10','cbsa10','metdiv10','ccflag10'],axis=1,inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [487]:
df_MA_sample_2000.drop(columns=['state','county','tract'],axis=1,inplace=True)

In [488]:
# this column exists in full count df
df_MA_sample_2000.drop(columns=['POP00SF3'],axis=1,inplace=True)

In [489]:
# drop columns that are not variables in 2010
df_MA_sample_2000.drop(columns=['DIS00','CNI16U00'],axis=1,inplace=True)

In [ ]:
## convert dtypes to floats

In [510]:
# fill in empty strings with 0's
df_MA_sample_2000['HINC00'] = pd.to_numeric(df_MA_sample_2000['HINC00'],errors='coerce')
df_MA_sample_2000['HINCW00'] = pd.to_numeric(df_MA_sample_2000['HINC00'],errors='coerce')
df_MA_sample_2000['HINCB00'] = pd.to_numeric(df_MA_sample_2000['HINCB00'],errors='coerce')
df_MA_sample_2000['HINCH00'] = pd.to_numeric(df_MA_sample_2000['HINCH00'],errors='coerce')
df_MA_sample_2000['HINCA00'] = pd.to_numeric(df_MA_sample_2000['HINCA00'],errors='coerce')
df_MA_sample_2000['MHMVAL00'] = pd.to_numeric(df_MA_sample_2000['MHMVAL00'],errors='coerce')
df_MA_sample_2000['MRENT00'] = pd.to_numeric(df_MA_sample_2000['MRENT00'],errors='coerce')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [519]:
df_MA_sample_2000.rename(columns={'TRTID10':'tractid'},inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


## Data pre-processing (2010 Census, Sample count)

In [529]:
df_2010_2 = pd.read_csv('LTDB_Std_2010_Sample.csv',sep=',', engine='python')
df_MA_sample_2010 = df_2010_2[(df_2010_2['statea']==25) & (df_2010_2['countya']==25)]

In [ ]:
# most of the variables in the sample files are derived from values in the full count files, so we are going
# to ignore those
# additionally, 2010 ACS has more variables - we are selecting only those that are in 2000 also

In [530]:
df_MA_sample_2010[df_MA_sample_2010['tractid']==25025060700]

,tractid,statea,countya,tracta,pnhwht12,pnhblk12,phisp12,pntv12,pasian12,phaw12,pindia12,pchina12,pfilip12,pjapan12,pkorea12,pviet12,p15wht12,p65wht12,p15blk12,p65blk12,p15hsp12,p65hsp12,p15ntv12,p65ntv12,p15asn12,p65asn12,pmex12,pcuban12,ppr12,pruanc12,pitanc12,pgeanc12,piranc12,pscanc12,pfb12,pnat12,p10imm12,prufb12,pitfb12,pgefb12,pirfb12,pscfb12,polang12,plep12,phs12,pcol12,punemp12,pflabf12,pprof12,pmanuf12,psemp12,pvet12,p65pov12,ppov12,pwpov12,pnapov12,pfmpov12,pbpov12,phpov12,papov12,pvac12,pown12,pmulti12,p30old12,p18und12,p60up12,p75up12,pmar12,pwds12,pfhh12,p10yrs12,ageblk12,agentv12,agewht12,agehsp12,india12,filip12,japan12,korea12,viet12,pop12,nhwht12,nhblk12,ntv12,hisp12,asian12,haw12,china12,a15wht12,a65wht12,a15blk12,a65blk12,a15hsp12,a65hsp12,a15ntv12,a65ntv12,ageasn12,a15asn12,a65asn12,mex12,pr12,cuban12,geanc12,iranc12,itanc12,ruanc12,fb12,nat12,itfb12,rufb12,ag5up12,irfb12,gefb12,scanc12,n10imm12,olang12,lep12,scfb12,ag25up12,dfmpov12,hh12,hinc12,hincb12,hincw12,hinch12,incpc12,ag18cv12,vet12,empclf12,dpov12,npov12,dbpov12,nbpov12,dnapov12,nnapov12,dwpov12,nwpov12,dhpov12,nhpov12,hhb12,hhw12,hhh12,hs12,col12,clf12,unemp12,dflabf12,flabf12,prof12,manuf12,semp12,hha12,hinca12,n65pov12,nfmpov12,napov12,dapov12,family12,hu12,vac12,ohu12,own12,rent12,dmulti12,mrent12,mhmval12,multi12,h30old12,h10yrs12,a18und12,a60up12,a75up12,ag15up12,12-Mar,wds12,fhh12
32382,25025060700,25,25,60700,23.620001,24.559999,43.0,0.0,5.04,0.0,0.0,5.04,0.0,0.0,0.0,0.0,10.98,12.77,38.59,3.42,29.389999,4.61,75.0,0.0,6.54,56.07,0.0,0.38,13.96,0.47,0.71,0.0,13.67,0.05,32.959999,16.5,14.95,0.0,0.0,0.0,0.0,0.0,62.560001,23.24,82.709999,4.63,11.55,56.68,4.74,4.35,0.0,1.7,2.59,30.360001,27.74,100.0,19.559999,25.1,33.880001,51.400002,3.23,0.0,85.669998,50.279999,33.470001,12.78,4.9,25.08,19.83,42.619999,65.889999,526,32,501,912,0,0,0,0,0,2121,501,521,0,912,107,0,107,55,64,203,18,268,42,24,0,107,7,60,0,296,8,0,290,15,10,699,350,0,0,1958,0,0,1,317,1225,455,0,1145,542,689,19481,19519,21838,18922,10089,1411,24,759,2121,644,526,132,32,32,501,139,912,309,130,191,289,947,53,857,99,958,543,36,33,0,71,11422.0,55,106,55,107,542,712,23,689,0,689,712,434,-999,610,358,454,710,271,104,1563,392,310,231


In [383]:
# recreate the list of columns from the 2000 dataset to match format 
columns = []
columns.append('tractid')
for column in df_MA_sample_2000.columns[1:]:
    column_without_year = column[:-2].lower()
    column_new_year = column_without_year +'12'
    columns.append(column_new_year)

In [400]:
# manually fix a couple that dont exactly match this format
columns[49] = 'hu12'
columns[51] = 'ohu12'
columns[21] = '12-Mar'

In [401]:
# subset df to match 2000 format
df_MA_sample_2010=df_MA_sample_2010[columns]

In [437]:
# merge so we can calculate % change

In [525]:
df_joined_sample = df_MA_sample_2000.merge(df_MA_sample_2010, on='tractid')

In [526]:
df_joined_sample.shape

(203, 133)

## Calculate change b/w censuses (sample count)

In [531]:
df_joined_sample.head()

,tractid,RUANC00,ITANC00,GEANC00,IRANC00,SCANC00,RUFB00,ITFB00,GEFB00,IRFB00,SCFB00,FB00,NAT00,N10IMM00,AG5UP00,OLANG00,LEP00,AG25UP00,HS00,COL00,AG15UP00,Mar-00,WDS00,CLF00,UNEMP00,DFLABF00,FLABF00,EMPCLF00,PROF00,MANUF00,SEMP00,AG18CV00,VET00,DPOV00,NPOV00,N65POV00,DFMPOV00,NFMPOV00,DWPOV00,NWPOV00,DBPOV00,NBPOV00,DNAPOV00,NNAPOV00,DHPOV00,NHPOV00,DAPOV00,NAPOV00,INCPC00,HU00SP,H30OLD00,OHU00SP,H10YRS00,DMULTI00,MULTI00,HINC00,HINCW00,HINCB00,HINCH00,HINCA00,MHMVAL00,MRENT00,HH00,HHW00,HHB00,HHH00,HHA00,ruanc12,itanc12,geanc12,iranc12,scanc12,rufb12,itfb12,gefb12,irfb12,scfb12,fb12,nat12,n10imm12,ag5up12,olang12,lep12,ag25up12,hs12,col12,ag15up12,12-Mar,wds12,clf12,unemp12,dflabf12,flabf12,empclf12,prof12,manuf12,semp12,ag18cv12,vet12,dpov12,npov12,n65pov12,dfmpov12,nfmpov12,dwpov12,nwpov12,dbpov12,nbpov12,dnapov12,nnapov12,dhpov12,nhpov12,dapov12,napov12,incpc12,hu12,h30old12,ohu12,h10yrs12,dmulti12,multi12,hinc12,hincw12,hincb12,hinch12,hinca12,mhmval12,mrent12,hh12,hhw12,hhb12,hhh12,hha12,ITANC00_POC,GEANC00_POC,IRANC00_POC,SCANC00_POC,RUFB00_POC,ITFB00_POC,GEFB00_POC,IRFB00_POC,SCFB00_POC,FB00_POC,NAT00_POC,N10IMM00_POC,AG5UP00_POC,OLANG00_POC,LEP00_POC,AG25UP00_POC,HS00_POC,COL00_POC,AG15UP00_POC,Mar-00_POC,WDS00_POC,CLF00_POC,UNEMP00_POC,DFLABF00_POC,FLABF00_POC,EMPCLF00_POC,PROF00_POC,MANUF00_POC,SEMP00_POC,AG18CV00_POC,VET00_POC,DPOV00_POC,NPOV00_POC,N65POV00_POC,DFMPOV00_POC,NFMPOV00_POC,DWPOV00_POC,NWPOV00_POC,DBPOV00_POC,NBPOV00_POC,DNAPOV00_POC,NNAPOV00_POC,DHPOV00_POC,NHPOV00_POC,DAPOV00_POC,NAPOV00_POC,INCPC00_POC,HU00SP_POC,H30OLD00_POC,OHU00SP_POC,H10YRS00_POC,DMULTI00_POC,MULTI00_POC,HINC00_POC,HINCW00_POC,HINCB00_POC,HINCH00_POC,HINCA00_POC,MHMVAL00_POC,MRENT00_POC,HH00_POC,HHW00_POC,HHB00_POC,HHH00_POC,HHA00_POC
0,25025000100,99.000000,528.000000,94.000000,636.000000,34.000000,24.000000,42.000000,7.0,55.000000,0.0,1033.000000,447.000000,417.00000,3752.000000,1290.000000,306.000000,2752.00000,1206.000000,1128.000000,3478.00000,1135.000000,416.000000,2446.000000,102.000000,1683.000000,1112.000000,2344.00000,982.000000,131.000000,266.00000,3414.000000,257.000000,3929.000000,529.000000,47.000000,700.000000,31.000000,2819.000000,433.000000,143.000000,13.0,31.000000,0.0,327.00000,56.0,423.000000,7.0,21999.00000,1614.000000,1314.000000,1563.000000,1064.000000,1614.000000,1379.000000,44630.00000,44630.00000,45000.0,57833.0000,63500.00000,222100.0000,898.000000,1562.000000,1227.000000,56.000000,108.000000,107.000000,61,384,110,471,108,29,18,0,15,14,1047,552,342,3358,1291,522,2563,1408,789,3258,902,557,2260,137,1587,988,2133,480,167,152,3192,176,3574,524,49,674,23,2415,448,141,9,0,0,629,13,353,52,27564,1681,1589,1510,1161,1681,1322,53571,54777,31759,46106,62115.0,348200,1348,1510,1184,38,143,136,-88.446970,308.510638,-82.704403,1285.294118,350.000000,-30.952381,157.142857,-100.000000,inf,-98.644724,134.228188,32.374101,-90.884861,160.310078,321.895425,-81.031977,112.520730,24.822695,-77.314549,187.048458,116.826923,-77.228128,2115.686275,-91.859774,42.715827,-57.849829,117.209776,266.412214,-37.218045,-95.547745,1142.023346,-95.520489,575.614367,1014.893617,-93.000000,2074.193548,-99.184108,457.736721,213.286713,984.615385,-70.967742,NaN,-100.0,1023.214286,-96.926714,4942.857143,-99.763626,1607.806691,27.929985,1.663468,41.917293,-28.066914,21.899927,-97.037867,20.033610,21.726667,-45.084986,-27.392126,-72.032868,38675.055679,-13.700384,23.064385,2014.285714,-64.814815,33.644860
1,25025000201,82.000000,272.000000,83.000000,967.000000,40.000000,12.000000,21.000000,14.0,219.000000,7.0,1001.000000,457.000000,446.00000,3774.000000,1007.000000,209.000000,2771.00000,808.000000,1312.000000,3526.00000,1055.000000,290.000000,2805.000000,54.000000,1747.000000,1342.000000,2751.00000,1351.000000,223.000000,173.00000,3455.000000,181.000000,3875.000000,337.000000,19.000000,680.000000,19.000000,2944.000000,210.000000,140.000000,43.0,0.000000,0.0,286.00000,40.0,304.000000,9.0,27675.00000,1574.000000,1458.000000,1543.000000,1128.000

In [532]:
# this is where 2000 data starts 
i = 1
# this is where 2010 data starts
j = 67
# stop once you get to final column
while j < 132:
    try:
        df_joined_sample[f'{df_joined_sample.columns[i]}_POC'] = df_joined_sample.iloc[:,1:].apply(lambda x: 100*
                                    ((x[df_joined_sample.columns[j]]-x[df_joined_sample.columns[i]])
                                     /x[df_joined_sample.columns[i]]),axis=1)
    except ZeroDivisionError:
        'NaN'
        
    i +=1
    j +=1

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# for 
i = 4
j = 45
# stop once you get to final column
while j < 86:
    df_joined[f'{df_joined.columns[i]}_POC'] = df_joined.iloc[:,4:].apply(lambda x: 100*
                                    ((x[df_joined.columns[j]]-x[df_joined.columns[i]])
                                     /x[df_joined.columns[i]]),axis=1)
    i +=1
    j +=1

In [533]:
df_joined_sample.to_csv('Sample_count_census_changes.csv')